In [22]:
import cv2 as cv
from moviepy import VideoFileClip
import numpy as np
from collections import deque
import imagehash
from PIL import Image

In [23]:
video_file = "USCvVT6thDrive.mp4"

In [24]:
# load video file to be analyized 
vid = cv.VideoCapture(video_file)
if vid.isOpened():
    # Read the first frame from the video
    ret, prev_frame = vid.read()
    ret, prev_frame = vid.read()
    ret, prev_frame = vid.read()
    # Create a pillow image from the numpy frame array
    first_frame_pil = Image.fromarray(cv.cvtColor(prev_frame, cv.COLOR_BGR2RGB))
    # Create a image hash of the first frame to be compare to check for camera angle
    first_frame_hash = imagehash.phash(first_frame_pil)

    # Reduce resolutions of the frame
    prev_frame = cv.resize(prev_frame, (0, 0), fx=0.5, fy=0.5)
    # Create empty que that will keep a running average of the mags of length 10
    motion_deque = deque(maxlen=30)
    snap = False
    correct_angle = True
    fps = vid.get(cv.CAP_PROP_FPS)
    time_of_snaps = []
    # Loop through the remain frames
    while ret:
        ret, next_frame = vid.read()
        
        if ret:
            # Reduce resolutions of the frame
            next_frame = cv.resize(next_frame, (0, 0), fx=0.5, fy=0.5)
            # Convert both frames to grayscale for analysis
            grayA = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)
            grayB = cv.cvtColor(next_frame, cv.COLOR_BGR2GRAY)

            # Take the flow of pixels from the prev frame to the next
            flow = cv.calcOpticalFlowFarneback(grayA, grayB, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            # Compute the magnitude and angle of the flow vectors
            magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

            # Take the median of the magnitude
            median_magnitude = np.median(magnitude)
            motion_deque.append(median_magnitude)

            # Snap deteched if the running average of the deque is over 0.1
            if np.mean(motion_deque) > 0.1 and not snap and correct_angle:
                snap = True
                print("Significant motion detected")
                # Get the time in milliseconds
                timestamp = vid.get(cv.CAP_PROP_POS_MSEC)
                # Change timestamp to seconds
                timestamp = timestamp / 1000
                time_of_snaps.append(timestamp)
        
            # Determine the end of the play
            if snap and np.mean(motion_deque) < 0.1:
                snap = False
            
            prev_frame = next_frame

# Release the video capture object and close all OpenCV windows
vid.release()
cv.destroyAllWindows()

Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion detected
Significant motion d

In [20]:
# Print the timestamps 
for time in time_of_snaps:
    print("Timestamp: ", time)

Timestamp:  2.3023000000000002
Timestamp:  37.320616666666666
Timestamp:  38.521816666666666
Timestamp:  41.30793333333334
Timestamp:  72.10536666666667
Timestamp:  77.44403333333334
Timestamp:  99.94985
Timestamp:  134.05058333333335
Timestamp:  149.5327166666667
Timestamp:  168.98548333333335
Timestamp:  171.78828333333334
Timestamp:  171.82165
Timestamp:  188.8887
Timestamp:  207.0068
Timestamp:  223.85696666666666
Timestamp:  226.3261
Timestamp:  295.3951
Timestamp:  295.5285666666667
Timestamp:  296.56293333333326
Timestamp:  297.6306666666667
Timestamp:  299.6994
Timestamp:  300.63366666666667
Timestamp:  301.1675333333333
Timestamp:  302.06843333333336
Timestamp:  302.65235
Timestamp:  312.27863333333335
Timestamp:  312.52888333333334
Timestamp:  314.63098333333335
Timestamp:  315.09811666666667
Timestamp:  317.9843333333333
Timestamp:  327.4938333333333
Timestamp:  348.86518333333333
Timestamp:  404.5041
Timestamp:  427.27685
Timestamp:  473.68988333333334
Timestamp:  489.47231

In [25]:

full_video = VideoFileClip(video_file)
# Determine each play and make a new video
i = 1
prev_time = 0
for start in time_of_snaps:
    # Skip if the start time is less than 5 seconds after the previous clip's start time
    # This is probably still the same play 
    if start - prev_time < 5:
        continue
    # Take 1 sec before snap and 5 seconds after as the timing for the clip
    # Define your desired buffer times
    pre_snap_buffer = 1  # 1 second before the snap
    post_snap_buffer = 5 # 5 seconds after the snap

    # Calculate the precise start and end of the play
    # Ensure the start time is not negative
    clip_start_time = max(0, start - pre_snap_buffer)
    # Ensure the end time is not greater than the length of the video
    clip_end_time = min(start + post_snap_buffer, full_video.duration)

    # Create a subclip using the calculated start and end times
    # .subclip(start_time, end_time)
    play_clip = full_video.subclipped(clip_start_time, clip_end_time)

    output_path = "USCOffsensevVTPlays/plays_6th_drive/play_clip_{}.mp4".format(i)
    play_clip.write_videofile(output_path, codec="libx264")
    prev_time = start
    i += 1

                                                                                 
                                                                                 
frame_index:   1%|          | 2/359 [21:40:50<3870:00:55, 39025.37s/it, now=None]

MoviePy - Building video USCOffsensevVTPlays/plays_6th_drive/play_clip_1.mp4.
MoviePy - Writing audio in play_clip_1TEMP_MPY_wvf_snd.mp3





                                                                                 
                                                                                 
frame_index:   1%|          | 2/359 [21:40:50<3870:01:29, 39025.46s/it, now=None]

MoviePy - Done.
MoviePy - Writing video USCOffsensevVTPlays/plays_6th_drive/play_clip_1.mp4



MoviePy - Done !
MoviePy - video ready USCOffsensevVTPlays/plays_6th_drive/play_clip_1.mp4


...

MoviePy - Writing audio in play_clip_35TEMP_MPY_wvf_snd.mp3





                                                                                 
                                                                                 
frame_index:   1%|          | 2/359 [21:41:56<3873:17:50, 39058.46s/it, now=None]

MoviePy - Done.
MoviePy - Writing video USCOffsensevVTPlays/plays_6th_drive/play_clip_35.mp4

















                                                                                 
                                                                                 
frame_index:   1%|          | 2/359 [21:42:27<3874:48:25, 39073.69s/it, now=None]

MoviePy - Done !
MoviePy - video ready USCOffsensevVTPlays/plays_6th_drive/play_clip_35.mp4
